In [1]:
%pylab inline
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from multiprocessing import Pool
from functools import partial 
import warnings
warnings.filterwarnings("ignore")
np.printoptions(precision=2)

# time line
T_min = 20
T_max = 80
T_y = 65
beta = 1/(1+0.02)
# States of the economy, GOOD or BAD, {1 : GOOD}, {0 : BAD}
S = [0,1]
# All the money amount are denoted in thousand dollars
earningShock = [0.8,1.2]
# Define transition matrix of economical states
# GOOD -> GOOD 0.8, BAD -> BAD 0.6 
P = np.array([[0.6, 0.4],[0.2, 0.8]])
# current risk free interest rate 
r_f = np.array([0.01 ,0.03])
# stock return depends on current and future econ states
r_m = np.array([[-0.2, 0.15],[-0.15, 0.2]])

    
#Define the utility function
def u(c):
    return np.log(c)
        

# Define deterministic function to captures the hump shape of life-cycle earnings.
ageTick = [20, 50, 60, 65, 80]
earning = [35, 100, 70, 35, 35]
linearFit = interp1d(ageTick, earning, kind = "linear")
detEarning = np.array(linearFit(np.linspace(T_min, T_max, T_max - T_min + 1)))

def y(age, s):
    if age <= T_y:
        return detEarning[age-T_min-1] * earningShock[s]
    else:
        return detEarning[age-T_min-1]

# Define the transtiion of state 
def transition(s, a):
    '''
        Input: Z is a tuple containing states variables: (w, s) which is the current wealth and econ state
               a is the action taken with contains variables: (c, b, k) which is the combination of consumption, bond and stock
        Output: the next possible states with corresponding probabilities
    '''
    c, b, k = a
    # depend on the current econ state s and future state s_next we have the following return on bond and stock 
    r_bond = r_f[s]
    r_stock = r_m[s]
    w_next = b*(1+r_bond) + k*(1+r_stock)
    return w_next

Populating the interactive namespace from numpy and matplotlib


In [2]:
# timeline is from age 15 - 80, use approximation for value function from age 15 - 79 by discretizing the state space
# since we know the terminal condition. Here XX is the grid point collection. Initiate the grid and corresponding value.

# This function is used to generate power grid 
def powspace(start, stop, power, num):
    start = np.power(start, 1/float(power))
    stop = np.power(stop, 1/float(power))
    return np.power(np.linspace(start, stop, num=num), power)

w_grid_size = 1000
w_lower = 0
w_upper = 120000
T = T_max - T_min
wgrid = powspace(w_lower, w_upper, 3, num = w_grid_size)
# wgrid = np.linspace(w_lower, w_upper, num = w_grid_size)

print("Shape of grid w grid: ", wgrid.shape)
# V use to store the value 
V_t = np.zeros((w_grid_size,len(S),T))
C_t = np.zeros((w_grid_size,len(S),T))
B_t = np.zeros((w_grid_size,len(S),T))
K_t = np.zeros((w_grid_size,len(S),T))
print("Shape of Value matrix: ", V_t.shape)

# Calculate terminal value function 
V_t[:, 0, T-1] = u(y(T_max,0) + wgrid)
V_t[:, 1, T-1] = u(y(T_max,1) + wgrid)
C_t[:, 0, T-1] = y(T_max,0) + wgrid
C_t[:, 1, T-1] = y(T_max,1) + wgrid
B_t[:, 0, T-1] = 0
B_t[:, 1, T-1] = 0
K_t[:, 0, T-1] = 0
K_t[:, 1, T-1] = 0

Shape of grid w grid:  (1000,)
Shape of Value matrix:  (1000, 2, 60)


In [3]:
from scipy.optimize import minimize, minimize_scalar
from quantecon.optimize.scalar_maximization import brent_max

# Definition of the value function, return of the funciton is the value of the function and the coresponding policy
theta_grid_size = 101
theta_grid = np.linspace(start = 0, stop = 1, num = 101)
vals = np.zeros(theta_grid_size)
bks = np.zeros(theta_grid_size)

def V(w, s, t, model):
    def obj(bk, theta):
        b = bk * theta
        k = bk * (1 - theta)
        c = y(t,s) + w - bk
        if c <= 0 or bk <= 0:
            return 9999999999999999
        w_next = transition(s, (c, b, k))
        V_next = np.array([model[state](w_next[state]) for state in S])
        return -(u(c) + beta * np.dot(V_next, P[s]))
    
    def f(the):
        res = minimize_scalar(obj, args = (the), method='brent')
        return [-res.fun, res.x]
    results = np.array(list(map(f, theta_grid)))
    vals = results[:,0]
    bks = results[:,1]
    index = np.argmax(vals)
    max_val = vals[index]
    max_the = theta_grid[index]
    max_bk = bks[index]
    return np.array([max_val, [max_bk*max_the, max_bk*(1-max_the)]])


In [4]:
pool = Pool()
# Set value to the grid by backward induction
for t in range(T_max - 1, T_min - 1, -1):
    print(t)
    tt = t-T_min-1
    cs = [interp1d(wgrid, V_t[:,0,tt+1], kind = "cubic", fill_value="extrapolate"),
          interp1d(wgrid, V_t[:,1,tt+1], kind = "cubic", fill_value="extrapolate")] 
    for s in S:
        f = partial(V, s = s, t = t, model = cs)
        results = np.array(pool.map(f, wgrid))
        V_t[:,s,tt] = results[:,0]
#        B_t[:,s,tt] = np.array([b for b in results[:,1]])
#         B_t[:,s,tt] = np.array([b[0] for b in results[:,1]])
#         K_t[:,s,tt] = np.array([k[1] for k in results[:,1]])
#         C_t[:,s,tt] = wgrid + y(t,s) - B_t[:,s,tt] - K_t[:,s,tt]
pool.close()

79


Process ForkPoolWorker-8:
Process ForkPoolWorker-1:
Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-2:
Process ForkPoolWorker-9:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-3:
  File "/home/hfsluser/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/hfsluser/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/hfsluser/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/hfsluser/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/hfsl

  File "/home/hfsluser/anaconda3/lib/python3.7/site-packages/scipy/optimize/optimize.py", line 1925, in optimize
    xa, xb, xc, fa, fb, fc, funcalls = self.get_bracket_info()
  File "/home/hfsluser/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-710f4e919b2c>", line 22, in f
    res = minimize_scalar(obj, args = (the), method='brent')
  File "<ipython-input-3-710f4e919b2c>", line 24, in V
    results = np.array(list(map(f, theta_grid)))
  File "/home/hfsluser/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-710f4e919b2c>", line 22, in f
    res = minimize_scalar(obj, args = (the), method='brent')
  File "/home/hfsluser/anaconda3/lib/python3.7/site-packages/scipy/optimize/optimize.py", line 2141, in _minimize_scalar_brent
    brent.optimize()
  File "/home/hfsluser/anaconda3/lib/python3.7/site-packages/sci

KeyboardInterrupt
  File "<ipython-input-3-710f4e919b2c>", line 18, in <listcomp>
    V_next = np.array([model[state](w_next[state]) for state in S])
  File "/home/hfsluser/anaconda3/lib/python3.7/site-packages/scipy/optimize/optimize.py", line 2141, in _minimize_scalar_brent
    brent.optimize()
  File "<__array_function__ internals>", line 6, in dot
  File "<ipython-input-3-710f4e919b2c>", line 24, in V
    results = np.array(list(map(f, theta_grid)))
  File "<ipython-input-3-710f4e919b2c>", line 24, in V
    results = np.array(list(map(f, theta_grid)))
  File "<ipython-input-3-710f4e919b2c>", line 18, in obj
    V_next = np.array([model[state](w_next[state]) for state in S])
  File "/home/hfsluser/anaconda3/lib/python3.7/site-packages/scipy/interpolate/polyint.py", line 79, in __call__
    y = self._evaluate(x)
  File "/home/hfsluser/anaconda3/lib/python3.7/site-packages/scipy/optimize/optimize.py", line 2141, in _minimize_scalar_brent
    brent.optimize()
  File "/home/hfsluser/ana

KeyboardInterrupt
  File "/home/hfsluser/anaconda3/lib/python3.7/site-packages/scipy/optimize/optimize.py", line 2141, in _minimize_scalar_brent
    brent.optimize()
  File "<ipython-input-3-710f4e919b2c>", line 17, in obj
    w_next = transition(s, (c, b, k))
  File "/home/hfsluser/anaconda3/lib/python3.7/site-packages/scipy/interpolate/_bsplines.py", line 359, in _evaluate
    self.k, xp, nu, extrapolate, out)
  File "/home/hfsluser/anaconda3/lib/python3.7/site-packages/scipy/optimize/optimize.py", line 1899, in get_bracket_info
    xa, xb, xc, fa, fb, fc, funcalls = bracket(func, args=args)
  File "/home/hfsluser/anaconda3/lib/python3.7/site-packages/scipy/interpolate/interpolate.py", line 649, in _call_spline
    return self._spline(x_new)
  File "<ipython-input-1-5dfe64165b12>", line 57, in transition
    w_next = b*(1+r_bond) + k*(1+r_stock)
  File "/home/hfsluser/anaconda3/lib/python3.7/site-packages/scipy/optimize/optimize.py", line 2141, in _minimize_scalar_brent
    brent.opt

KeyboardInterrupt: 

  File "/home/hfsluser/anaconda3/lib/python3.7/site-packages/scipy/interpolate/_bsplines.py", line 359, in _evaluate
    self.k, xp, nu, extrapolate, out)
KeyboardInterrupt
KeyboardInterrupt


In [ ]:
from matplotlib import pyplot as plt
plt.plot(C_t[100,1,:])
plt.plot(C_t[200,1,:])
plt.plot(C_t[300,1,:])
plt.plot(C_t[400,1,:])
plt.plot(C_t[500,1,:])
plt.plot(C_t[600,1,:])

In [ ]:
plt.plot(B_t[100,1,:])
plt.plot(B_t[200,1,:])
plt.plot(B_t[300,1,:])
plt.plot(B_t[400,1,:])
plt.plot(B_t[500,1,:])
plt.plot(B_t[900,1,:])

In [ ]:
plt.plot(K_t[100,1,:])
plt.plot(K_t[200,1,:])
plt.plot(K_t[300,1,:])
plt.plot(K_t[400,1,:])
plt.plot(K_t[500,1,:])
plt.plot(K_t[600,1,:])

### Simulation Part

In [ ]:
import quantecon as qe
def mc_sample_path(P, x0=None, sample_size=1000):

    # set up
    P = np.asarray(P)
    X = np.empty(sample_size, dtype=int)

    # Convert each row of P into a cdf
    n = len(P)
    P_dist = [np.cumsum(P[i, :]) for i in range(n)]

    # draw initial state, defaulting to 0
    if x0 is not None:
        X_0 = qe.random.draw(np.cumsum(x0))
    else:
        X_0 = 0

    # simulate
    X[0] = X_0
    for t in range(sample_size - 1):
        X[t+1] = qe.random.draw(P_dist[X[t]])

    return X

def action(t, w, s):
    c = interp1d(wgrid, C_t[:,s,t], kind = "linear", fill_value = "extrapolate")(w)
    b = interp1d(wgrid, B_t[:,s,t], kind = "linear", fill_value = "extrapolate")(w)
    k = interp1d(wgrid, K_t[:,s,t], kind = "linear", fill_value = "extrapolate")(w)
    return (c,b,k)
    
# Define the transtiion of state 
def fixTransition(w, s, s_next, a):
    c, b, k = a
    # collect possible next state (w_next, s_next) with probability prob
    Z_next = []
    prob = []
    # depend on the current econ state s and future state s_next we have the following return on bond and stock 
    r_bond = r_f[int(s)]
    r_stock = r_m(s,s_next)
    w_next = b*(1+r_bond) + k*(1+r_stock)
    return w_next

In [ ]:
def simulation(simulations = 1000):
    for sim in range(simulations):
        if sim%100 == 0:
            print(sim)
        # simulate an agent age 15 starting with wealth of 10
        w = 20
        wealth = []
        Consumption = []
        Bond = []
        Stock = []
        econState = mc_sample_path(P, sample_size=60)
        for t in range(len(econState)-1):
            wealth.append(w)
            s = econState[t]
            s_next = econState[t+1]
            a = action(t, w, s)
            Consumption.append(a[0])
            Bond.append(a[1])
            Stock.append(a[2])
            w = fixTransition(w,s,s_next, a)
        # dictionary of lists  
        dictionary = {'wealth': wealth,
                      'Consumption': Consumption, 
                      'Bond': Bond, 
                      'Stock': Stock}  
        if sim == 0:
            df = pd.DataFrame(dictionary) 
        else:
            df = df + pd.DataFrame(dictionary) 
            
    return df 

In [ ]:
df = simulation(simulations = 1000)

In [ ]:
(df/1000).plot()